In [ ]:
%load_ext tensorboard

Check if running in colab, if yes then clone the git repository and navigate into the correct directory

In [ ]:
# This checks if
# (a) the code is running in google colab
# (b) the repository hasn't been cloned yet
import sys
from pathlib import Path
if "google.colab" in sys.modules and not Path.cwd().name == "license_plate_extraction":
    !git clone https://github.com/cxan96/license_plate_detection
    %cd license_plate_detection
    !git submodule init
    !git submodule update
    %cd src/license_plate_extraction

In [ ]:
import tensorflow as tf
from tensorflow.keras import losses
import numpy as np
from data_reader import get_image_paths_from_directory, make_dataset_from_image_paths
from preprocessing import bounding_box_in_pixel
from visualization_tools import show_image
import settings
import models
import datetime

In [ ]:
tf.test.gpu_device_name()

Read all the available images into a list of paths:

In [ ]:
image_directory_eu = settings.DATA_DIR / "eu_cars+lps"
image_directory_br = settings.DATA_DIR / "br_cars+lps"
image_directory_us = settings.DATA_DIR / "us_cars+lps"
image_directory_ro = settings.DATA_DIR / "ro_cars+lps"

# make a list with all the image paths
all_image_paths = np.array(
    [
        *get_image_paths_from_directory(image_directory_eu),
        *get_image_paths_from_directory(image_directory_br),
        *get_image_paths_from_directory(image_directory_us),
        *get_image_paths_from_directory(image_directory_ro),
    ]
)

num_images = len(all_image_paths)

print(f"NUM_IMAGES = {num_images}")

Split the list of all image paths into a list of training paths and a list of test paths:

In [ ]:
TRAINING_SET_SIZE = int(num_images * 0.8)
train_indices = np.random.choice(num_images, TRAINING_SET_SIZE, replace=False)

image_path_list_train = all_image_paths[train_indices]
image_path_list_test = np.delete(all_image_paths, train_indices)

Create the training set as well as the test set

In [ ]:
TARGET_IMG_HEIGHT = 448
TARGET_IMG_WIDTH = 448
BATCH_SIZE = 32

dataset_train = make_dataset_from_image_paths(
    image_path_list_train,
    target_img_height=TARGET_IMG_HEIGHT,
    target_img_width=TARGET_IMG_WIDTH,
)
dataset_train = dataset_train.batch(BATCH_SIZE)

dataset_test = make_dataset_from_image_paths(
    image_path_list_test,
    target_img_height=TARGET_IMG_HEIGHT,
    target_img_width=TARGET_IMG_WIDTH,
)
dataset_test = dataset_test.batch(BATCH_SIZE)

Get the model:

In [ ]:
model = models.tiny_yolo_inspired(
    input_height=TARGET_IMG_HEIGHT, input_width=TARGET_IMG_WIDTH, num_channels=3
)

model.summary()

Set up the optimizer

In [ ]:
model.compile(optimizer="adam", loss=losses.MSE)

Launch TensorBoard:

In [ ]:
%tensorboard --logdir $settings.LOG_DIR

Fit the model to the training data:

In [ ]:
# add Tensorboard callback
log_dir = settings.LOG_DIR / (
    "training_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1
)

model.fit(
    dataset_train,
    epochs=10,
    validation_data=dataset_test,
    callbacks=[tensorboard_callback],
)

Show the predictions on the test set

In [ ]:
example_list = list(dataset_test.as_numpy_iterator())

for cur_example in example_list:
    cur_image_batch = cur_example[0]
    cur_prediction_batch = model.predict(cur_image_batch)
    
    for cur_image, cur_prediction in zip(cur_image_batch, cur_prediction_batch):
        print(cur_prediction)
        
        show_image(
            cur_image.astype(int),
            bounding_box=bounding_box_in_pixel(
                cur_prediction,
                img_height=TARGET_IMG_HEIGHT,
                img_width=TARGET_IMG_WIDTH,
            ),
        )